# Lesson 3: Enable Logging

### Import all needed packages

In [45]:
import boto3
import json
import os

bedrock = boto3.client('bedrock', region_name="us-west-2")

In [63]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [64]:
from helpers.CloudWatchHelper import CloudWatch_Helper
cloudwatch = CloudWatch_Helper()

In [65]:
log_group_name = '/my/amazon/bedrock/logs'

In [66]:
cloudwatch.create_log_group(log_group_name)

Log group '/my/amazon/bedrock/logs' already exists.


In [67]:
loggingConfig = {
    'cloudWatchConfig': {
        'logGroupName': log_group_name,
        'roleArn': os.environ['LOGGINGROLEARN'],
        'largeDataDeliveryS3Config': {
            'bucketName': os.getenv('BucketName'),
            'keyPrefix': 'amazon_bedrock_large_data_delivery',
        }
    },
    's3Config': {
        'bucketName': os.getenv('BucketName'),
        'keyPrefix': 'amazon_bedrock_logs',
    },
    'textDataDeliveryEnabled': True,
}

In [68]:
bedrock.put_model_invocation_logging_configuration(loggingConfig=loggingConfig)

{'ResponseMetadata': {'RequestId': 'e8b8993b-5ca8-444f-89bd-ecb16a1ece11',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 24 Nov 2024 14:33:23 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e8b8993b-5ca8-444f-89bd-ecb16a1ece11'},
  'RetryAttempts': 0}}

In [69]:
bedrock.get_model_invocation_logging_configuration()

{'ResponseMetadata': {'RequestId': '3da2b3c2-8eb3-4862-b0b6-5ccfef202793',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 24 Nov 2024 14:33:24 GMT',
   'content-type': 'application/json',
   'content-length': '483',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3da2b3c2-8eb3-4862-b0b6-5ccfef202793'},
  'RetryAttempts': 0},
 'loggingConfig': {'cloudWatchConfig': {'logGroupName': '/my/amazon/bedrock/logs',
   'roleArn': 'arn:aws:iam::092413168457:role/CloudWatchES-Course',
   'largeDataDeliveryS3Config': {'bucketName': 'elephantscale-public-temp',
    'keyPrefix': 'amazon_bedrock_large_data_delivery'}},
  's3Config': {'bucketName': 'elephantscale-public-temp',
   'keyPrefix': 'amazon_bedrock_logs'},
  'textDataDeliveryEnabled': True,
  'imageDataDeliveryEnabled': True,
  'embeddingDataDeliveryEnabled': True}}

In [70]:
bedrock_runtime = boto3.client('bedrock-runtime', region_name="us-west-2")

In [71]:
prompt = "Write an article about the fictional planet Foobar."

kwargs = {
    "modelId": "amazon.titan-text-express-v1",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(
        {
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        }
    )
}

response = bedrock_runtime.invoke_model(**kwargs)
response_body = json.loads(response.get('body').read())

generation = response_body['results'][0]['outputText']

print(generation)


Foobar is a fictional planet that is featured in a series of science fiction novels and films. The planet is set in the distant future and is inhabited by a diverse range of alien species.

One of the most interesting aspects of Foobar is its geography. The planet is a vast, desert wasteland with towering sand dunes and vast stretches of barren land. The climate is harsh and inhospitable, with temperatures reaching over 100 degrees Fahrenheit during the day and dropping to below freezing at night. The only source of water on the planet is a small oasis in the center of the desert, which is protected by a powerful energy field that prevents anyone from entering or leaving.

The inhabitants of Foobar are a diverse group of alien species, each with its own unique culture and beliefs. The most prominent of these species is the Barbaricons, a race of giant, muscle-bound warriors who rule the planet with an iron fist. The Barbaricons are fiercely protective of their territory and are consta

In [72]:
cloudwatch.print_recent_logs(log_group_name)

Permissions are correctly set for Amazon Bedrock logs.
-------------------------



To review the logs within the AWS console, please use the following link to reference the steps outlined in the video:

In [77]:
from IPython.display import HTML
aws_url = os.environ['AWS_CONSOLE_URL']

In [78]:
HTML(f'<a href="{aws_url}" target="_blank">GO TO AWS CONSOLE</a>')
